In [57]:
import gc
import re
import string
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import classification_report, accuracy_score,f1_score
from transformers import AutoModel
from transformers import BertModel, BertTokenizer
from torch.utils.data import Dataset , DataLoader
from sklearn.preprocessing import LabelEncoder
import datasets
from tqdm import tqdm
from sklearn.metrics import classification_report

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [58]:
valid_dataset = datasets.load_dataset('social_bias_frames',split="validation[:7000]")
test_dataset = datasets.load_dataset('social_bias_frames',split="test[:9000]")

In [59]:
class HateDataset(Dataset):
    def __init__(self, dataset, tokenizer, model, max_length):
        dataset = dataset.to_pandas()
        dataset = dataset[dataset['offensiveYN'] != '']
        dataset.loc[dataset['offensiveYN'] == '0.5', 'offensiveYN'] = '1.0'
        dataset = dataset.groupby(['post','offensiveYN']).size().reset_index(name='counts')
        dataset = dataset.sort_values('counts', ascending=False).drop_duplicates('post')
        dataset = datasets.Dataset.from_pandas(dataset)
        label_encoder = LabelEncoder()
        self.label = label_encoder.fit_transform(dataset['offensiveYN'])
        self.post = dataset['post']
        self.tokenizer = tokenizer
        self.model = model
        self.max_length=max_length

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        tokenized_post = self.tokenizer(self.post[idx], return_tensors='pt',max_length=self.max_length, padding='max_length', truncation=True)
        with torch.no_grad():
            all_layers = self.model(input_ids=tokenized_post['input_ids'], attention_mask=tokenized_post['attention_mask'], output_hidden_states=True)
        return self.label[idx], all_layers.hidden_states

In [60]:
class HateDataset2(Dataset):
    def __init__(self, dataset, tokenizer, model):
        dataset = dataset.to_pandas()
        dataset = dataset[dataset['offensiveYN'] != '']
        dataset.loc[dataset['offensiveYN'] == '0.5', 'offensiveYN'] = '1.0'
        dataset = dataset.groupby(['post','offensiveYN']).size().reset_index(name='counts')
        dataset = dataset.sort_values('counts', ascending=False).drop_duplicates('post')
        dataset = datasets.Dataset.from_pandas(dataset)
        label_encoder = LabelEncoder()
        self.label = label_encoder.fit_transform(dataset['offensiveYN'])
        self.post = dataset['post']
        self.tokenizer = tokenizer
        self.model = model

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        tokenized_post = self.tokenizer(self.post[idx], return_tensors='pt',max_length=50, padding='max_length', truncation=True)
        with torch.no_grad():
            model_output = self.model(**tokenized_post)
        last_hidden_state = model_output.last_hidden_state
        return self.label[idx], last_hidden_state

In [61]:
class BERT_FC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BERT_FC, self).__init__()
        self.fc1 = nn.Linear(50 * input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(-1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [62]:
class BERT_Arch_CNN(nn.Module):
    def __init__(self):
        super(BERT_Arch_CNN, self).__init__()
        self.conv = nn.Conv2d(in_channels=13, out_channels=13, kernel_size=(3, 768), padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(624, 2)
        self.flat = nn.Flatten()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, all_layers):
        x = torch.transpose(torch.cat(tuple([t.unsqueeze(0) for t in all_layers]), 0), 0, 1)
        torch.cuda.empty_cache()
        x = self.pool(self.dropout(self.relu(self.conv(self.dropout(x)))))
        x = self.fc(self.dropout(self.flat(self.dropout(x))))
        return x

In [63]:
class BERT_Arch_CNN_BiLSTM(nn.Module):
    def __init__(self):
        super(BERT_Arch_CNN_BiLSTM, self).__init__()
        self.conv = nn.Conv2d(in_channels=13, out_channels=13, kernel_size=(3, 514), padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=1, stride=1)
        self.bilstm = nn.LSTM(input_size = 768, hidden_size = 256, num_layers = 2, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(1300, 2)
        self.flat = nn.Flatten()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, all_layers):
        x = torch.transpose(torch.cat(tuple([t.unsqueeze(0) for t in all_layers]), 0), 0, 1)
        x, _ = self.bilstm(x.squeeze())
        torch.cuda.empty_cache()
        x = self.dropout(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = self.flat(x)
        x = self.dropout(x)
        x = x.view(-1)
        x = self.fc(x)
        return x.unsqueeze(0)

In [64]:
def evaluate(model,test_dataloader):
    model = model.to(device)
    model.eval()
    test_running_loss = 0.0
    test_all_predictions = []
    test_all_labels = []
    criterion=nn.CrossEntropyLoss()

    with torch.no_grad():
        for labels, all_layers in tqdm(test_dataloader, desc=f"Testing"):
            labels = torch.tensor(labels)
            for i in range(len(all_layers)):
                all_layers[i] = all_layers[i].squeeze().unsqueeze(0)
            all_layers = torch.stack(all_layers)
            all_layers = all_layers.to(device)
            labels=labels.to(device)
            outputs = model(all_layers)
            one_hot_targets = torch.zeros(1, 2).to(device)
            one_hot_targets.scatter_(1, labels.unsqueeze(1), 1)
            loss = criterion(outputs.squeeze(), one_hot_targets.squeeze())

            test_running_loss += loss.item()

            predicted = torch.argmax(outputs, dim=1)
            test_all_predictions.extend(predicted.cpu().tolist())
            test_all_labels.extend(labels.cpu().tolist())

        test_epoch_loss = test_running_loss / len(test_dataloader)
        test_epoch_accuracy = accuracy_score(test_all_labels, test_all_predictions)
        test_epoch_f1 = f1_score(test_all_labels, test_all_predictions, average='macro')
        print(f"Testing, Loss: {test_epoch_loss:.4f}, Accuracy: {test_epoch_accuracy:.4f}, F1: {test_epoch_f1:.4f}")
        print(classification_report(test_all_labels,test_all_predictions))

In [65]:
def evaluate2(model,test_dataloader):
    model = model.to(device)
    model.eval()
    running_loss = 0.0
    test_all_predictions = []
    test_all_labels = []
    criterion=nn.CrossEntropyLoss()

    for labels, inputs in tqdm(test_dataloader, desc=f"Testing"):
        inputs = torch.tensor(inputs[0])
        labels = torch.tensor([labels[0]])
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs.squeeze())
        one_hot_targets = torch.zeros(1, 2).to(device)
        one_hot_targets.scatter_(1, labels.unsqueeze(1), 1)
        loss = criterion(outputs, one_hot_targets.squeeze())
        running_loss += loss.item()
        predicted = torch.argmax(outputs.unsqueeze(0), dim=1)
        test_all_predictions.extend(predicted.cpu().tolist())
        test_all_labels.extend(labels.cpu().tolist())

    test_loss = running_loss / len(test_dataloader)
    test_accuracy = accuracy_score(test_all_labels, test_all_predictions)
    test_f1 = f1_score(test_all_labels, test_all_predictions, average='macro')

    print(f"Testing, Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}")
    print(classification_report(test_all_labels,test_all_predictions))

In [66]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [67]:
test_data = HateDataset(test_dataset,tokenizer,bert_model,50)
test_dataloader = DataLoader(test_data , batch_size=1 , shuffle=False)

model = BERT_Arch_CNN()
model.load_state_dict(torch.load('Model_BERT_CNN_10.pth'))
evaluate(model,test_dataloader)

Testing:   0%|          | 0/2407 [00:00<?, ?it/s]<ipython-input-64-c4ca5b0ab59e>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)
Testing: 100%|██████████| 2407/2407 [06:46<00:00,  5.93it/s]

Testing, Loss: 0.6527, Accuracy: 0.6647, F1: 0.6622
              precision    recall  f1-score   support

           0       0.58      0.69      0.63      1003
           1       0.75      0.64      0.69      1404

    accuracy                           0.66      2407
   macro avg       0.66      0.67      0.66      2407
weighted avg       0.68      0.66      0.67      2407



In [68]:
test_data = HateDataset(test_dataset,tokenizer,bert_model,100)
test_dataloader = DataLoader(test_data , batch_size=1 , shuffle=False)

model = BERT_Arch_CNN_BiLSTM()
model.load_state_dict(torch.load('Model_BERT_CNN_BiLSTM_10.pth'))
evaluate(model,test_dataloader)

Testing:   0%|          | 0/2407 [00:00<?, ?it/s]<ipython-input-64-c4ca5b0ab59e>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)
Testing: 100%|██████████| 2407/2407 [10:37<00:00,  3.78it/s]

Testing, Loss: 0.5570, Accuracy: 0.7270, F1: 0.7204
              precision    recall  f1-score   support

           0       0.67      0.69      0.68      1003
           1       0.77      0.75      0.76      1404

    accuracy                           0.73      2407
   macro avg       0.72      0.72      0.72      2407
weighted avg       0.73      0.73      0.73      2407



In [69]:
test_data = HateDataset2(test_dataset,tokenizer,bert_model)
test_dataloader = DataLoader(test_data , batch_size=1 , shuffle=False)

model = BERT_FC(768,128,2)
model.load_state_dict(torch.load('Model_FC_10.pt'))
evaluate2(model,test_dataloader)

Testing:   0%|          | 0/2407 [00:00<?, ?it/s]<ipython-input-65-da7c046ea1dc>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs[0])
Testing: 100%|██████████| 2407/2407 [06:39<00:00,  6.03it/s]

Testing, Loss: 0.6219, Accuracy: 0.6897, F1: 0.6654
              precision    recall  f1-score   support

           0       0.67      0.50      0.58      1003
           1       0.70      0.82      0.76      1404

    accuracy                           0.69      2407
   macro avg       0.68      0.66      0.67      2407
weighted avg       0.69      0.69      0.68      2407

